In [145]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
import unicodedata
import re

Team name constants

In [146]:
team_tags_dico = {
    "SCL" : ["scl","scald"],
    "IWG" : ["iwg"]
}

In [147]:
driver_options = webdriver.FirefoxOptions()
driver_options.add_argument("--headless")
driver = webdriver.Firefox(options=driver_options)

In [148]:
driver.get("")
time.sleep(1)

In [149]:
def getBlue_picks_bans(soup,name):
    picks = soup.find(attrs={"class": "roomPickColumn blue"}).get_text("|")
    picks = picks.split("|")
    bans = [x['alt'] for x in soup.find(attrs={"class": "roomBanRow blue"}).find_all('img')]
    return picks,bans, name

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html,"html.parser")

getBlue_picks_bans(soup,"blue")

In [151]:
def getRed_picks_bans(soup,name):
    picks = soup.find(attrs={"class": "roomPickColumn red"}).get_text("|")
    picks = picks.split("|")
    bans = [x['alt'] for x in soup.find(attrs={"class": "roomBanRow red"}).find_all('img')]
    return picks, bans, name

In [ ]:
getRed_picks_bans(soup,"red")

In [153]:
def detect_team_in_name(name,team_reference_dict, side) :

    for tag, keywords in team_reference_dict.items():
        for  word in keywords :
            if word in name.lower()  :
                return tag
    return side

In [154]:
def extract_date(name):

    match = re.search(r'\d', name)
    if match :
        digit_index = match.start()
        return name[digit_index:]
    return "NaT"

In [155]:
def get_side_by_tag(soup,team_reference_dict):
    #TODO ATTENTION, il faut récupérer le nom de l'équipe, et si dans le nom qui contient Scald il y a des chiffres, alors
    # récupérer chiffre et split pour obtenir date de la game et numéro de la game
    blue_text = unicodedata.normalize("NFKD",soup.find(attrs={"class" : "roomReadyBackground roomReadyBackgroundblue"}).previous_sibling.get_text())
    red_text = unicodedata.normalize("NFKD",soup.find(attrs={"class" : "roomReadyBackground roomReadyBackgroundred"}).previous_sibling.get_text())
    
    blue_team = detect_team_in_name(blue_text, team_reference_dict, "Blue")
    red_team = detect_team_in_name(red_text,team_reference_dict, "Red")

    if blue_team == "SCL" :
        game_date = extract_date(blue_text)
    elif red_team == "SCL" :
        game_date = extract_date(red_text)
    else :
        game_date = "NaT"
    return blue_team, red_team, game_date

In [ ]:
get_side_by_tag(soup,team_tags_dico)

In [138]:
def scraping_draft(draft_url,team_reference_dict):

    driver.get(draft_url)
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    blue_name , red_name = get_side_by_tag(soup,team_reference_dict)

    blue = getBlue_picks_bans(soup,blue_name)
    red = getRed_picks_bans(soup,red_name)
    
    
    return blue, red